<h1> Goal of this book is to combine text detection and image processing (+character segmentation) with OCR model </h1>

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import cv2
import imutils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import keras_ocr
from keras.models import load_model


In [ ]:
detector = keras_ocr.detection.Detector()
model = load_model("textReadModel(printed).keras") # model trained on printed text
model.summary()

In [ ]:
old_model = load_model("textReadModel(handwritten).keras") # model trained on handwritten dataset
model.summary()

In [ ]:
#label binarizer for digits and characters to allow inverse transform (to see the predicted text as a text)
lb = LabelBinarizer()
lb.fit_transform(["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F",
        "G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U",
        "V","W","X","Y","Z","a","b","c","d","e","f","g","h","i","j",
        "k","l","m","n","o","p","q","r","s","t","u","v","w","x","y",
        "z"])
lb.classes_

Recognition and Post-Processing <br>
The sort contours function is used to get the correct order of individual characters for correct output extraction. In this case for extracting a single word, a left to right sorting of individual characters is needed.
The get letters function fetches the list of letters and get word function gets the words.

In [ ]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
    key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [ ]:
def get_word(letters):
    word = "".join(letters)
    return word

Method to read each character in the image and convert it to text
- Procedure of img processing need to be improved to achieve better recognition results

In [ ]:
def ocr_read_cnt(letters, temp, contours, model, img_size):
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w > 10 and h > 10:
            cv2.rectangle(temp,(x,y),(x+w,y+h),(0,255,0),1)
            #predict letter in this rectangle
            # plt.imshow(temp)
            # plt.show()
            letter = temp[y:y+h,x:x+w]
            letter = cv2.resize(letter, (img_size, img_size))
            letter = cv2.cvtColor(letter, cv2.COLOR_BGRA2GRAY)
            letter = cv2.GaussianBlur(letter, (3, 3), 0)
            ret, letter = cv2.threshold(letter, 127, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)
            letter = cv2.bilateralFilter(letter, 3, 100, 100, borderType=cv2.BORDER_CONSTANT)
            # morph
            kernel = np.ones((1,1), np.uint8)
            letter = cv2.erode(letter, kernel, iterations=1)
            letter = cv2.dilate(letter, kernel, iterations=1)
            letter = cv2.bitwise_not(letter)
                
            letter = letter.astype("float32") / 255.0
            letter = np.expand_dims(letter, axis=2)
            letter = np.expand_dims(letter, axis=0)
            # plt.imshow(letter.reshape(32,32))
            # plt.show()
            prediction = model.predict(letter)
            prediction = lb.inverse_transform(prediction)
#           print(prediction)
            letters.append(prediction[0])
            

In [ ]:
def detectText (img):
    # get predictions of text boxes
    predictions = detector.detect(images=[img])[0]

    # draw boxes around text
    image = keras_ocr.tools.drawBoxes(img, predictions, (0,255,0), 0)

    # show image
    plt.imshow(image)
    plt.show()

    # process each detected box as a separate image
    #for i in range(len(predictions)):
    for i in range(3): #fore testing to check only first 3 words
        letters = []
        old_letters = []
        temp = keras_ocr.tools.warpBox(img, predictions[i], margin=2) # crop each word
        plt.imshow(temp)
        plt.show()

        #img processing
        gray = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
        tresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

        #contours
        contours, hierarchy = cv2.findContours(tresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        contours = sort_contours(contours, method="left-to-right")[0]

        #remove first contour (bounding box of entire image)
        contours = contours[1:]

        #draw bounding boxes around contours of letters from left to right and predict with models trained on different datasets
        ocr_read_cnt(letters, temp, contours, model, 28)
        word = get_word(letters)
        print('printed model: ', word)

        ocr_read_cnt(old_letters, temp, contours, old_model, 32)
        word = get_word(old_letters)
        print('handrwitten model: ', word)

        # save each image (uncomment to save)
      #  tf.keras.utils.save_img('images/temp_text_{}.jpg'.format(i), temp)

In [ ]:
# test with an image of a product label from phone camera
imgTest = keras_ocr.tools.read('images/product_test_2.jpg')
detectText(imgTest)